In [6]:
import cv2, time, requests, re, numpy as np
from typing import Optional
from ultralytics import YOLO
from sort.sort import Sort
from util import get_car, reader     

BACKEND_URL = "http://visaonestbackend-env.eba-25r2yw9h.us-east-2.elasticbeanstalk.com/cars/detect"

CHAR_TO_DIGIT = {'O': '0', 'I': '1', 'J': '3', 'A': '4', 'G': '6', 'S': '5'}
DIGIT_TO_CHAR = {v: k for k, v in CHAR_TO_DIGIT.items()}

PATTERN_OLD      = re.compile(r'^[A-Z]{3}\d{4}$')        
PATTERN_MERCOSUL = re.compile(r'^[A-Z]{3}\d[A-Z]\d{2}$') 

def normalize_plate(text: str) -> Optional[str]:
    """
    Normaliza string de placa:
    - Remove prefixos como 'BR', espaços, traços, pontos.
    - Corrige confusões comuns do OCR.
    - Valida formato de placas brasileiras.
    """
    if not text:
        return None

    # Remove todos os caracteres não alfanuméricos e espaços
    cleaned = re.sub(r'[^A-Za-z0-9]', '', text.upper().replace(' ', ''))
    if cleaned.startswith("BR"):
        cleaned = cleaned[2:]

    if not cleaned:
        return None

    # Corrige confusões comuns
    chars = list(cleaned)
    for i in range(len(chars)):
        ch = chars[i]
        if i in (0, 1, 4, 5, 6):  # posições esperadas de letras
            chars[i] = DIGIT_TO_CHAR.get(ch, ch)
        elif i in (2, 3):        # posições esperadas de números
            chars[i] = CHAR_TO_DIGIT.get(ch, ch)

    plate = ''.join(chars)

    if len(plate) != 7:
        return None

    if PATTERN_OLD.fullmatch(plate) or PATTERN_MERCOSUL.fullmatch(plate):
        return plate
    return None

def sanitize_plate_for_sending(plate: str) -> str:
    """
    Remove qualquer caractere inválido da placa antes do envio,
    garantindo que será enviada como uma única string alfanumérica válida.
    """
    return re.sub(r'[^A-Za-z0-9]', '', plate.upper().strip())


def extract_plate_text(thresh_img):
    """
    Itera pelas detecções do EasyOCR até encontrar uma placa válida.
    """
    for _, text, score in reader.readtext(thresh_img, detail=1):
        plate = normalize_plate(text)
        print(f"    OCR raw='{text}' -> normalizado='{plate}' (score={score:.2f})")
        if plate:
            return plate, score
    return None, None

def send_detection(track_id: int, plate: str):
    """
    Usa a string limpa da placa para envio.
    """
    cleaned_plate = sanitize_plate_for_sending(plate)
    payload = {
        "trackId": track_id,
        "plate": cleaned_plate
    }
    print(f"\nEnviando payload: {payload}")
    try:
        r = requests.post(BACKEND_URL, json=payload, timeout=5)
        print(f"{r.status_code}: {r.text}")
        r.raise_for_status()
    except requests.RequestException as e:
        print(f"Falha HTTP: {e}")


def preprocess_crop(crop):
    """
    Aumenta a resolução e aplica threshold adaptativo para melhorar o OCR.
    """
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    thresh = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 35, 15)
    return thresh

def main():
    coco = YOLO("yolov8n.pt")
    lp   = YOLO("/home/messyas/ml/jetson/placas-model/models/runs/yolov8n_lp/weights/best.pt")
    tracker = Sort()

    img = cv2.imread("/home/messyas/Downloads/nomeimagem.jpg")
    if img is None:
        print("Imagem não encontrada.")
        return

    try:
        while True:
            # 1) Detecta veículos
            veh_boxes = [
                b for b in coco(img)[0].boxes.data.tolist()
                if int(b[5]) in [2, 3, 5, 7]
            ]
            tracks = tracker.update(np.array([[x1, y1, x2, y2, s] for x1, y1, x2, y2, s, _ in veh_boxes]))

            # 2) Detecta placas
            for box in lp(img)[0].boxes.data.tolist():
                x1, y1, x2, y2, score, _ = box
                *_, car_id = get_car(box, tracks)
                if car_id == -1:
                    continue

                crop   = img[int(y1):int(y2), int(x1):int(x2)]
                plate, conf = extract_plate_text(preprocess_crop(crop))

                if plate:
                    print(f"Placa final: {plate}")
                    send_detection(int(car_id), plate)
                else:
                    print("Nenhuma placa válida detectada")

            time.sleep(5)
    except KeyboardInterrupt:
        print("Encerrado pelo usuário.")

if __name__ == "__main__":
    main()


0: 384x640 1 person, 1 car, 34.2ms
Speed: 0.5ms preprocess, 34.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 6.0ms
Speed: 0.5ms preprocess, 6.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
    OCR raw='WIY6g88' -> normalizado='WIY6G88' (score=0.42)
Placa final: WIY6G88

Enviando payload: {'trackId': 3, 'plate': 'WIY6G88'}
200: {"message":"Placa WIY6G88 cadastrada com sucesso"}

0: 384x640 1 person, 1 car, 6.3ms
Speed: 0.7ms preprocess, 6.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 4.0ms
Speed: 0.5ms preprocess, 4.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
    OCR raw='WIY6g88' -> normalizado='WIY6G88' (score=0.42)
Placa final: WIY6G88

Enviando payload: {'trackId': 3, 'plate': 'WIY6G88'}
200: {"message":"Placa WIY6G88 já cadastrada"}

0: 384x640 1 person, 1 car, 5.2ms
Speed: 0.7ms preprocess, 5.2ms inference, 1.2ms postprocess per im